In [1]:
import numpy as np
from numpy import float64
from numpy.typing import NDArray, ArrayLike
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sci

In [ ]:
class Rede_Neural:
    def __init__(self, qtd_neuronios_camada, seed) -> None:
        self.camadas: int = len(qtd_neuronios_camada)
        self.neuronios_camada: list[int] = qtd_neuronios_camada
        self.pesos = [self.pesos_iniciais(seed)]
        self.seed = seed

    def ativacao(self, z: ArrayLike) -> NDArray:
        """
        Aplica a função de ativação definida em uma lista ou np.array. Atualmente é leaky ReLU, com 0.01*z se z < 0
        """
        return np.where(z > 0, z, 0.01 * z)

    def derivada_ativacao(self, z: ArrayLike) -> NDArray:
        """
        Aplica a derivada da função de ativação definida em uma lista ou np.array. Atualmente é leaky ReLU, com 0.01 se z < 0
        """
        return np.where(z > 0, 1, 0.01)

    def pesos_iniciais(self, seed: int) -> list[NDArray[float64]]:
        """
        Usa a distribuição de Kaiming para criar pesos iniciais para ReLU usando a seed dada para o rng.\n
        W(camada) ~ N(0, sqrt(2 / neuronios[camada]))
        """
        pesos_lista = []
        np.random.seed(seed)
        for i in range(self.camadas - 1):
            pesos = np.random.normal(
                0,
                np.sqrt(2/self.neuronios_camada[i]),
                size=(self.neuronios_camada[i+1], self.neuronios_camada[i]+1)
            )
            pesos_lista.append(pesos)
        return pesos_lista

    def forward_pass(self, entrada: ArrayLike, pesos: list[NDArray]) -> list[dict[str, NDArray]]:
        """
        Aplica a rotina de multiplicar pelos pesos e aplicar função de ativação para todas as camadas, retornando os valores pré-ativação e pós-ativação de cada camada.

        Parameters
        ----------
        entrada: ArrayLike
            É a lista de valores de entrada da rede, pode ser uma lista mesmo ou qualquer ArrayLike, já que é transformada em NDArray dentro da função.

        pesos: list[NDArray]
            São os pesos que se quer usar na rede. Usar self.pesos[-1] para última época

        Returns
        -------
        valores_camadas: list[dict]
            Para uma camada qualquer ``i``, os valores de ``dicionário valores_camadas[i]`` são:

            - ``"z"``: NDArray or None\n
                Valores não ativados da camada
            - ``"a"``: NDArray\n
                Valores ativados da camada
        """
        valores_camadas = []
        a = np.array(entrada)
        valores_camadas.append({"z": None, "a": a})

        for W in pesos:
            z = W @ np.append(a, 1) # adiciona bias
            a = self.ativacao(z)
            valores_camadas.append({"z": z, "a": a})
        
        return valores_camadas
    
    def previsao(self, entrada: ArrayLike, epoca: int) -> NDArray:
        """Retorna os valores previstos pela rede, os valores ativados da última camada, para a época desejada. Usar -1 para última época"""
        camadas  = self.forward_pass(entrada, self.pesos[epoca])
        previsao = camadas[-1]["a"]
        return previsao

    def perda(self, previsao: NDArray, real: ArrayLike) -> float64:
        """
        Calcula a função de perda da rede, atualmente usando 0.5 MSE mas poderia ser com penalidades e
        para função de valor máximo talvez seja melhor usar uma função como softmax
        """
        residuos: NDArray = previsao - real
        mse = np.mean(residuos ** 2)
        perda = 0.5 * mse
        return perda
    
    def derivada_perda(self, previsao: NDArray, real: ArrayLike) -> NDArray:
        """
        Calcula a derivada da função de perda, lembrar que a derivada de abs(x) = sign(x)
        """
        residuos: NDArray = previsao - real
        return residuos
    
    def calcular_gradiente(self, entrada: ArrayLike, real: ArrayLike, pesos: list[NDArray]) -> list[NDArray]:
        """Calcula o gradiente dos pesos para os pesos dados, retornando uma lista de NDArrays correspondente ao gradiente de cada camada."""
        valores_camadas = self.forward_pass(entrada, pesos)
        previsao = valores_camadas[-1]["a"]
        
        # alocacao de listas
        delta = [None] * (self.camadas - 1)
        gradiente = [None] * (self.camadas - 1)
    
        # delta da ultima camada
        erro_saida = self.derivada_perda(previsao, real)
        delta[-1] = erro_saida * self.derivada_ativacao(valores_camadas[-1]["z"])

        # gradiente da ultima camada
        a_anterior = np.append(valores_camadas[-2]["a"], 1)
        gradiente[-1] = np.outer(delta[-1], a_anterior)

        # gradiente das camadas ocultas
        for camada in reversed(range(self.camadas - 2)):
            # delta da camada
            W_sem_bias = pesos[camada + 1][:, :-1] # descarta coluna do bias
            delta[camada] = (W_sem_bias.T @ delta[camada + 1]) * self.derivada_ativacao(valores_camadas[camada + 1]["z"])
            
            # gradiente da camada
            a_anterior = np.append(valores_camadas[camada]["a"], 1)
            gradiente[camada] = np.outer(delta[camada], a_anterior)

        return gradiente

    def treinar_uma_epoca(self, taxa_aprendizado: float, tamanho_lotes: int, entradas_treino: list[ArrayLike], saidas_treino: list[ArrayLike]) -> None:
        """Calcula os gradientes e corrige os pesos de acordo com os parâmetros dados, salvando os novos pesos em self.pesos."""
        # embaralha os indices
        n = len(entradas_treino)
        indices = np.arange(n)
        np.random.shuffle(indices)
        novos_pesos = [W.copy() for W in self.pesos[-1]] # usa a ultima epoca

        # percorre os lotes
        for inicio in range(0, n, tamanho_lotes):
            fim = inicio + tamanho_lotes
            indices_lote = indices[inicio:fim]
            gradiente_acumulado = [np.zeros_like(W) for W in novos_pesos]

            # calcula o gradiente e soma no gradiente acumulado
            for i in indices_lote:
                entrada = np.array(entradas_treino[i])
                saida_real = np.array(saidas_treino[i])

                gradiente = self.calcular_gradiente(entrada, saida_real, novos_pesos)

                for j in range(self.camadas - 1):
                    gradiente_acumulado[j] += gradiente[j]

            # tira o gradiente medio
            gradiente_medio = [g / len(indices_lote) for g in gradiente_acumulado]
            
            # corrige os pesos
            for i in range(self.camadas-1):
                novos_pesos[i] -= taxa_aprendizado * gradiente_medio[i]
        
        self.pesos.append(novos_pesos)

    def treinar(self, taxa_aprendizado: float, tamanho_lotes: int, entradas_treino: list[ArrayLike], saidas_treino: list[ArrayLike], quantidade_epocas: int) -> None:
        """Chama self.treinar_uma_epoca ``quantidade_epocas`` vezes"""
        np.random.seed(self.seed)
        for epoca in range(quantidade_epocas):
            self.treinar_uma_epoca(taxa_aprendizado, tamanho_lotes, entradas_treino, saidas_treino)

In [3]:
rede = Rede_Neural([2,3,2], 42)

In [4]:
rede.previsao([1,2],-1)

array([-0.00734121, -0.04552614])